#設定

In [ ]:
%%capture
!pip install transformers fugashi ipadic

In [ ]:
import os
import random
import unicodedata
import tensorflow as tf
import numpy as np

from tensorboard.plugins import projector
from transformers import BertJapaneseTokenizer, TFBertModel

In [ ]:
%load_ext tensorboard

In [ ]:
MODEL_NAME = "cl-tohoku/bert-base-japanese-whole-word-masking"

#データセット

In [ ]:
!wget https://www.rondhuit.com/download/ldcc-20140209.tar.gz
!tar -zxf ldcc-20140209.tar.gz

In [ ]:
filepaths = tf.io.gfile.glob("./text/*/*-*.txt")

In [ ]:
texts = []
titles = []
for path in filepaths:
    with open(path, "r") as f:
        content = f.readlines()
        titles.append(content[2].replace("\n", ""))
        texts.append(unicodedata.normalize(
            "NFKC", "".join(content[3:])
        ).replace("\n", ""))

In [ ]:
tokenizer = BertJapaneseTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
MAX_LENGTH = 64

encodings = tokenizer(
    texts, max_length=MAX_LENGTH,
    padding="max_length", truncation=True,
    return_tensors="tf"
)

#学習なしのBertで埋め込む

In [ ]:
bert = TFBertModel.from_pretrained(MODEL_NAME)

全部埋め込むと見にくくなるので、最初の1000件に限定する

In [ ]:
output = bert(
    [encodings["input_ids"][:1000], encodings["attention_mask"][:1000], encodings["token_type_ids"][:1000]]
)

In [ ]:
os.mkdir("./logs")

ラベルはメタデータとしてtsvで保存する

In [ ]:
with open(os.path.join("logs", "metadata.tsv"), "w") as f:
    for title in titles[:1000]:
        f.write("{}\n".format(title))

埋め込みベクトルを変数として、チェックポイントで保存する

In [ ]:
checkpoint = tf.train.Checkpoint(embedding=tf.Variable(
    output.pooler_output.numpy()
))
checkpoint.save(os.path.join("logs", "embedding.ckpt"))

In [ ]:
config = projector.ProjectorConfig()
embedding = config.embeddings.add()
# The name of the tensor will be suffixed by `embedding`
embedding.tensor_name = "embedding"
embedding.metadata_path = 'metadata.tsv'
projector.visualize_embeddings("./logs", config)

In [ ]:
%tensorboard --logdir ./logs